In [12]:
import random
import time

# Salvando dados no arquivo
nome_arquivo = "knapsack_data.txt"
with open(nome_arquivo, 'w') as arquivo:
    arquivo.write("5\n")  # número de itens
    arquivo.write("10\n")  # Capacidade Q
    arquivo.write("1 1 1 5 5\n")  # Pesos
    arquivo.write("1 2 3 7 8\n")  # Lucros

# Carregando dados do arquivo
with open(nome_arquivo, 'r') as arquivo:
    linhas = arquivo.readlines()

n = int(linhas[0].strip())  # Número de itens
Q = int(linhas[1].strip())  # Capacidade da mochila
pesos = list(map(int, linhas[2].strip().split()))
lucros = list(map(int, linhas[3].strip().split()))

# Criar a representação da solução como uma lista de números binários
def criar_solucao(n, Q, pesos, lucros, limite_tempo=1, estrategia="aleatoria"):
    inicio = time.time()
    solucao = [0] * n
    peso_total = 0
    
    if estrategia == "inteligente":
        itens = sorted(range(n), key=lambda i: lucros[i] / pesos[i], reverse=True)
    else:
        itens = list(range(n))
        random.shuffle(itens)
    
    for i in itens:
        if time.time() - inicio >= limite_tempo:
            break
        if peso_total + pesos[i] <= Q:
            solucao[i] = 1
            peso_total += pesos[i]
    
    return solucao

# Função de avaliação
def avaliar_solucao(solucao, lucros, pesos, Q, penalizar_inviavel=True):
    lucro_total = sum([lucros[i] * solucao[i] for i in range(len(solucao))])
    peso_total = sum([pesos[i] * solucao[i] for i in range(len(solucao))])
    
    if peso_total > Q and penalizar_inviavel:
        penalidade = (peso_total - Q) * 10  # Penalidade mais significativa
        return lucro_total - penalidade
    return lucro_total

# Função de busca local para melhorar a solução
def busca_local(solucao_inicial, lucros, pesos, Q, limite_tempo=1):
    inicio = time.time()
    melhor_solucao = solucao_inicial[:]
    melhor_lucro = avaliar_solucao(melhor_solucao, lucros, pesos, Q)
    
    while time.time() - inicio < limite_tempo:
        for i in range(len(solucao_inicial)):
            nova_solucao = melhor_solucao[:]
            nova_solucao[i] = 1 - nova_solucao[i]  # Tenta inverter o item i
            novo_lucro = avaliar_solucao(nova_solucao, lucros, pesos, Q)
            if novo_lucro > melhor_lucro:
                melhor_solucao = nova_solucao
                melhor_lucro = novo_lucro
    
    return melhor_solucao

# Gerar 10 soluções para cada heurística
def gerar_solucoes(n, Q, pesos, lucros, num_solucoes=10):
    solucoes_aleatorias = []
    solucoes_inteligentes = []
    
    for _ in range(num_solucoes):
        solucao_aleatoria = criar_solucao(n, Q, pesos, lucros, estrategia="aleatoria")
        solucao_inteligente = criar_solucao(n, Q, pesos, lucros, estrategia="inteligente")
        solucao_inteligente = busca_local(solucao_inteligente, lucros, pesos, Q)  # Melhorar a solução inteligente
        solucoes_aleatorias.append((solucao_aleatoria, avaliar_solucao(solucao_aleatoria, lucros, pesos, Q)))
        solucoes_inteligentes.append((solucao_inteligente, avaliar_solucao(solucao_inteligente, lucros, pesos, Q)))
    
    return solucoes_aleatorias, solucoes_inteligentes

# Comparar heurísticas aleatória e inteligente
solucoes_aleatorias, solucoes_inteligentes = gerar_solucoes(n, Q, pesos, lucros)

# Comparar avaliações médias das soluções aleatórias e inteligentes
def comparar_heuristicas(solucoes_aleatorias, solucoes_inteligentes):
    media_aleatoria = sum([solucao[1] for solucao in solucoes_aleatorias]) / len(solucoes_aleatorias)
    media_inteligente = sum([solucao[1] for solucao in solucoes_inteligentes]) / len(solucoes_inteligentes)
    
    if media_aleatoria > media_inteligente:
        return f"A heurística aleatória teve melhor desempenho com um lucro médio de {media_aleatoria:.2f}."
    elif media_inteligente > media_aleatoria:
        return f"A heurística inteligente teve melhor desempenho com um lucro médio de {media_inteligente:.2f}."
    else:
        return "Ambas as heurísticas tiveram desempenho igual."
    
# Saída dos resultados de cada solução antes de mostrar a comparação
def mostrar_resultados(solucoes_aleatorias, solucoes_inteligentes):
    print("Soluções Aleatórias:")
    for solucao in solucoes_aleatorias:
        print(f"Solução: {solucao[0]} | Lucro: {solucao[1]}")

    print("\nSoluções Inteligentes:")
    for solucao in solucoes_inteligentes:
        print(f"Solução: {solucao[0]} | Lucro: {solucao[1]}")

# Mostrar os resultados antes de comparar
mostrar_resultados(solucoes_aleatorias, solucoes_inteligentes)

# Saída da comparação
print(comparar_heuristicas(solucoes_aleatorias, solucoes_inteligentes))

Soluções Aleatórias:
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 1, 0] | Lucro: 13
Solução: [1, 1, 1, 1, 0] | Lucro: 13
Solução: [1, 1, 1, 1, 0] | Lucro: 13
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 1, 0] | Lucro: 13
Solução: [1, 1, 1, 0, 1] | Lucro: 14

Soluções Inteligentes:
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
A heurística inteligente teve melhor desempenho com um lucro médio de 14.00.


In [3]:
import random
import time

# Lê os dados do arquivo
def ler_dados(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()

    n = int(linhas[0].strip())  # Número de itens
    Q = int(linhas[1].strip())  # Capacidade da mochila
    pesos = list(map(int, linhas[2].strip().split()))
    lucros = list(map(int, linhas[3].strip().split()))

    return n, Q, pesos, lucros

# Exemplo de uso:
n, Q, pesos, lucros = ler_dados("knapsack_data.txt")
solucoes_aleatorias, solucoes_inteligentes = gerar_solucoes(n, Q, pesos, lucros)

# Criar a representação da solução como uma lista de números binários
def criar_solucao_aleatoria(n, Q, pesos, limite_tempo=1):
    inicio = time.time()
    solucao = [0] * n
    peso_total = 0
    while time.time() - inicio < limite_tempo:
        for i in range(n):
            if peso_total + pesos[i] <= Q:
                solucao[i] = random.choice([0, 1])
                peso_total = sum([pesos[j] * solucao[j] for j in range(n)])
            else:
                break
    return solucao

# Heurística inteligente para criar solução inicial
def criar_solucao_inteligente(n, Q, pesos, lucros, limite_tempo=1):
    inicio = time.time()
    solucao = [0] * n
    itens = sorted(range(n), key=lambda i: lucros[i] / pesos[i], reverse=True)  # Ordenar por razão lucro/peso
    peso_total = 0
    
    for i in itens:
        if time.time() - inicio >= limite_tempo:
            break
        if peso_total + pesos[i] <= Q:
            solucao[i] = 1
            peso_total += pesos[i]
    
    return solucao

# Função de avaliação
def avaliar_solucao(solucao, lucros, pesos, Q, penalizar_inviavel=True):
    lucro_total = sum([lucros[i] * solucao[i] for i in range(len(solucao))])
    peso_total = sum([pesos[i] * solucao[i] for i in range(len(solucao))])
    
    if peso_total > Q and penalizar_inviavel:
        penalidade = peso_total - Q
        return lucro_total - penalidade * sum(lucros)
    return lucro_total

# Gerar 10 soluções para cada heurística
def gerar_solucoes(n, Q, pesos, lucros, num_solucoes=10):
    solucoes_aleatorias = []
    solucoes_inteligentes = []
    
    for _ in range(num_solucoes):
        solucao_aleatoria = criar_solucao_aleatoria(n, Q, pesos)
        solucao_inteligente = criar_solucao_inteligente(n, Q, pesos, lucros)
        solucoes_aleatorias.append((solucao_aleatoria, avaliar_solucao(solucao_aleatoria, lucros, pesos, Q)))
        solucoes_inteligentes.append((solucao_inteligente, avaliar_solucao(solucao_inteligente, lucros, pesos, Q)))
    
    return solucoes_aleatorias, solucoes_inteligentes

# Comparar heurísticas aleatória e inteligente
solucoes_aleatorias, solucoes_inteligentes = gerar_solucoes(n, Q, pesos, lucros)

# Mostrar resultados
print("Soluções Aleatórias:")
for solucao in solucoes_aleatorias:
    print(f"Solução: {solucao[0]} | Lucro: {solucao[1]}")

print("\nSoluções Inteligentes:")
for solucao in solucoes_inteligentes:
    print(f"Solução: {solucao[0]} | Lucro: {solucao[1]}")


Soluções Aleatórias:
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15
Solução: [0, 0, 0, 1, 1] | Lucro: 15

Soluções Inteligentes:
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
Solução: [1, 1, 1, 0, 1] | Lucro: 14
